In [17]:
import pandas as pd
import numpy as np
import re

In [15]:
df_raw = pd.read_csv("/Users/yoran/Documents/Advanced_Simulation/Bangladesh_Assignment/data/raw/_roads.tsv", sep='\t')

df_bridges_raw = pd.read_excel('/Users/yoran/Documents/Advanced_Simulation/Bangladesh_Assignment/data/raw/BMMS_overview.xlsx')

/var/folders/04/yj5x8lbx49sc1jbf3gyzwr3h0000gn/T/ipykernel_43399/1161747910.py:1: DtypeWarning: Columns (571,574,577,580,583,586,589,592,595,598,601,604,607,610,613,616,619,622,625,628,631,634,637,640,643,646,649,652,655,658,661,664,667,670,673,676,679,682,685,688,691,694,697,700,703,706,709,712,715,718,721,724,727,730,733,736,739,742,745,748,751,754,757,760,763,766,769,772,775,778,781,784,787,790,793,796,799,802,805,808,811,814,817,820,823,826,829,832,835,838,841,844,847,850,853,856,859,862,865,868,871,874,877,880,883,886,889,892,895,898,901,904,907,910,913,916,919,922,925,928,931,934,937,940,943,946,949,952,955,958,961,964,967,970,973,976,979,982,985,988,991,994,997,1000,1003,1006,1009,1012,1015,1018,1021,1024,1027,1030,1033,1036,1039,1042,1045,1048,1051,1054,1057,1060,1063,1066,1069,1072,1075,1078,1081,1084,1087,1090,1093,1096,1099,1102,1105,1108,1111,1114,1117,1120,1123,1126,1129,1132,1135,1138,1141,1144,1147,1150,1153,1156,1159,1162,1165,1168,1171,1174,1177,1180,1183,1186,1189,119

In [12]:
# 2. Inspect the columns
print("Columns found:", df_bridges_raw.columns.tolist())

# 3. Quick Clean (Standardize names)
# It's good practice to strip whitespace from column names immediately
df_bridges_raw.columns = df_bridges_raw.columns.str.strip()

# Check the first few rows to ensure the header was picked up correctly
print(df_bridges_raw.head())

Columns found: ['road', 'km', 'type', 'LRPName', 'name', 'length', 'condition', 'structureNr', 'roadName', 'chainage', 'width', 'constructionYear', 'spans', 'zone', 'circle', 'division', 'sub-division', 'lat', 'lon', 'EstimatedLoc']
  road      km              type  LRPName               name  length  \
0   N1   1.800       Box Culvert  LRP001a                  .   11.30   
1   N1   4.925       Box Culvert  LRP004b                  .    6.60   
2   N1   8.976  PC Girder Bridge  LRP008b  Kanch pur Bridge.  394.23   
3   N1  10.880       Box Culvert  LRP010b   NOYAPARA CULVERT    6.30   
4   N1  10.897       Box Culvert  LRP010c     ADUPUR CULVERT    6.30   

  condition  structureNr                                           roadName  \
0         A       117861  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...   
1         A       117862  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...   
2         A       119889  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...   
3         A       

In [24]:
import pandas as pd
import numpy as np
import re

# 1. LOAD & CLEAN (Standard Procedure)
print("Loading raw data...")
df_raw = pd.read_csv('/Users/yoran/Documents/Advanced_Simulation/Bangladesh_Assignment/data/raw/_roads.tsv', sep='\t', low_memory=False)

# Parse Wide -> Long
new_rows = []
for i in range(1, len(df_raw.columns), 3):
    if i+2 >= len(df_raw.columns): break
    chunk = df_raw.iloc[:, [0, i, i+1, i+2]].copy()
    chunk.columns = ['road', 'lrp', 'lat', 'lon']
    chunk = chunk.dropna(subset=['lrp'])
    new_rows.append(chunk)

df_long = pd.concat(new_rows, ignore_index=True)

# Extract Chainage for Sorting
def get_chainage(val):
    val = str(val).upper()
    if 'LRPS' in val: return 0.0
    match = re.search(r'(\d+\.?\d*)', val)
    if match: return float(match.group(1))
    return np.nan

df_long['chainage'] = df_long['lrp'].apply(get_chainage)
df_long['lat'] = pd.to_numeric(df_long['lat'], errors='coerce')
df_long['lon'] = pd.to_numeric(df_long['lon'], errors='coerce')

# Filter & Sort
mask_lat = (df_long['lat'] > 20) & (df_long['lat'] < 27)
mask_lon = (df_long['lon'] > 88) & (df_long['lon'] < 93)
df_clean = df_long[mask_lat & mask_lon].copy()
df_clean = df_clean.sort_values(by=['road', 'chainage'])
df_clean = df_clean.drop_duplicates(subset=['road', 'chainage'])

print(f"Cleaned data points: {len(df_clean)}")

# ---------------------------------------------------------
# 2. PIVOT BACK TO WIDE FORMAT (The "Restorer")
# ---------------------------------------------------------
# We need to reconstruct: road, lrp1, lat1, lon1, lrp2...

print("Reconstructing wide format...")
grouped = df_clean.groupby('road')
rows = []
max_len = 0

for road_name, group in grouped:
    # Get values as lists (They are already sorted by chainage!)
    lrps = group['lrp'].tolist()
    lats = group['lat'].tolist()
    lons = group['lon'].tolist()

    current_len = len(lrps)
    if current_len > max_len: max_len = current_len

    # Create a dictionary for this row
    row_data = {'road': road_name}
    for i in range(current_len):
        idx = i + 1  # 1-based index (lrp1, lrp2...)
        row_data[f'lrp{idx}'] = lrps[i]
        row_data[f'lat{idx}'] = lats[i]
        row_data[f'lon{idx}'] = lons[i]

    rows.append(row_data)

# Create DataFrame
df_wide = pd.DataFrame(rows)

# Enforce Column Order (road, lrp1, lat1, lon1, lrp2...)
# This is critical because dicts are unordered
col_order = ['road']
for i in range(1, max_len + 1):
    col_order.extend([f'lrp{i}', f'lat{i}', f'lon{i}'])

# Reindex arranges columns correctly and fills missing spots with NaN
df_wide = df_wide.reindex(columns=col_order)

# ---------------------------------------------------------
# 3. SAVE AS TSV
# ---------------------------------------------------------
output_path = '/Users/yoran/Documents/Advanced_Simulation/Bangladesh_Assignment/data/raw/_roads.tsv' # Overwriting original
# output_path = 'data/processed/_roads_cleaned_wide.tsv' # Safer option

df_wide.to_csv(output_path, sep='\t', index=False)
print(f"Success! Saved cleaned wide format to {output_path}")
print("Columns:", df_wide.columns.tolist()[:10])

Loading raw data...
Cleaned data points: 21596
Reconstructing wide format...
Success! Saved cleaned wide format to /Users/yoran/Documents/Advanced_Simulation/Bangladesh_Assignment/data/raw/_roads.tsv
Columns: ['road', 'lrp1', 'lat1', 'lon1', 'lrp2', 'lat2', 'lon2', 'lrp3', 'lat3', 'lon3']


In [25]:
df_wide

,road,lrp1,lat1,lon1,lrp2,lat2,lon2,lrp3,lat3,lon3,...,lon503,lrp504,lat504,lon504,lrp505,lat505,lon505,lrp506,lat506,lon506
0,N1,LRPS,23.706028,90.443333,LRP001,23.702139,90.451972,LRP002,23.697889,90.460583,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,N101,LRPS,23.454139,91.212861,LRP001,23.462944,91.211806,LRP002,23.463111,91.221055,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,N102,LRPS,23.478972,91.118194,LRP001,23.486722,91.113306,LRP002,23.493527,91.106917,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,N103,LRPS,23.957028,91.115528,LRP001,23.961917,91.113611,LRP002,23.970610,91.111583,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,N104,LRPS,23.009667,91.399416,LRP001,23.009306,91.389805,LRP002,23.009528,91.380360,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872,Z8910,LRPS,22.674722,90.408889,LRP001,22.675583,90.417166,LRP002,22.674527,90.426666,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873,Z8913,LRPS,22.396083,90.688666,LRP001,22.387250,90.689389,LRP002,22.378249,90.689083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
874,Z8915,LRPS,22.589389,90.619472,LRP001,22.589694,90.623360,LRP002,22.590194,90.633083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
875,Z8916,LRPS,22.625499,90.661722,LRP001,22.620305,90.668999,LRP002,22.616389,90.677249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
